In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import numpy as np
import scipy.io as sio
from numpy import expand_dims
from numpy import asarray
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from skimage.filters import gabor_kernel
from scipy import ndimage as nd        
import matplotlib
import matplotlib.pyplot as plt    
from numpy.linalg import norm
from keras.optimizers import SGD, Adam
import graphviz
import pydot

In [2]:
data_path = 'E:/Sushree/Dataset/'

img_path = os.path.join(data_path,'Animals_with_Attributes2/JPEGImages/')
print(img_path)

print(len(os.listdir(img_path)))

def get_imlist(path, option):
    if option == 'jpg':
        return[os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

width, height, ch = 224, 224, 3



E:/Sushree/Dataset/Animals_with_Attributes2/JPEGImages/
50


In [ ]:

kernels = []
for theta in range(2):
    theta = theta / 4. * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            kernel = gabor_kernel(frequency, theta = theta, sigma_x = sigma, sigma_y = sigma)
            kernels.append(kernel)
            
def define_power(image, kernel):
    # Normalize images for better comparison.
    image = (image - image.mean()) / image.std()
    #print(np.shape(image))
    return np.sqrt(nd.convolve(image, np.real(kernel), mode='wrap')**2 + nd.convolve(image, np.imag(kernel), mode='wrap')**2)
            
def compute_gabor_feats(image, kernels):
    feature = np.zeros((len(kernels), image.shape[0], image.shape[1]))
    for k, kernel in enumerate(kernels):
        res_pow = define_power(image, kernel)
        feature[k] = res_pow
        #feature = np.add(feature, res_pow)                
    return feature  

def get_imlist(path, option):
    if option == 'jpg':
        return[os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]

width, height, ch = 224, 224, 3

def compute_feature_dataset(img_path, width, height, option, kernels):
    folder_list = os.listdir(img_path)
    num_classes = len(folder_list)
    gabor_feature = []
    for i in range(len(folder_list)):
        print(i)
        img_list = get_imlist(os.path.join(img_path, folder_list[i]), option)
        for j in range(len(img_list)):
            img = image.load_img(img_list[j], color_mode="grayscale", target_size = (width, height))
            img = asarray(img)
            gabor_feat = compute_gabor_feats(img, kernels)
            #gabor_feat = np.expand_dims(gabor_feat, axis=0)
            gabor_feature.append(gabor_feat)
    gabor_feature = np.array(gabor_feature, dtype = np.float32)         
    return gabor_feature 


gabor_feature = compute_feature_dataset(img_path, width, height, 'jpg', kernels)
print(np.shape(gabor_feature))    

np.save('C:/Users/Admin/Sushree_Codes/Sush/Results/gabor_feature_8_kernels.npy', gabor_feature)


In [3]:

split_path = os.path.join(data_path,'data/xlsa17/data/AWA2/att_splits.mat')
matcontent = sio.loadmat(split_path)
trainval_loc = matcontent['trainval_loc'].squeeze() - 1

#print(matcontent)

print(trainval_loc, len(trainval_loc))

gabor_feature = np.load('C:/Users/Admin/Sushree_Codes/Sush/Results/gabor_feature_8_kernels.npy')
print(gabor_feature.shape)

print(np.shape(gabor_feature))
print(trainval_loc, trainval_loc.shape)

train_gabor_feature = []
for i in range (len(trainval_loc)):
    train_gabor_feat = gabor_feature[trainval_loc[i]]
    train_gabor_feat = train_gabor_feat.reshape(np.prod(train_gabor_feat.shape[0:]))
    train_gabor_feature.append(train_gabor_feat)

train_gabor_feature = np.array(train_gabor_feature, dtype = np.float32) 
print(np.shape(train_gabor_feature))



[21482 11452 21448 ...  7992 10388  5160] 23527
(37322, 8, 224, 224)
(37322, 8, 224, 224)
[21482 11452 21448 ...  7992 10388  5160] (23527,)
(23527, 401408)


In [4]:

# performing preprocessing part
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_t = sc.fit_transform(train_gabor_feature)

from sklearn.decomposition import KernelPCA

transformer = KernelPCA(n_components = 4096, kernel = 'poly')
train_gabor_feature_transformed = transformer.fit_transform(X_t)
print(train_gabor_feature_transformed, train_gabor_feature_transformed.shape)

[[ 9.6899301e-01  9.8722791e-03 -1.6094959e-01 ...  1.4629345e-02
  -4.3485691e-03  4.5536556e-03]
 [-1.5022326e-01 -1.0419858e-01  6.6463433e-02 ...  1.3577098e-02
  -2.2116555e-03 -4.4724456e-04]
 [ 6.1230052e-01  4.7015655e-01 -1.7734341e-01 ...  2.6094878e-02
   1.4556493e-02 -5.9137302e-03]
 ...
 [ 6.6859767e-02 -3.9864212e-01 -2.4804704e-01 ...  2.2252318e-03
  -2.0175776e-03  9.7561618e-03]
 [-7.0907420e-01  3.4373632e-01 -4.1183540e-01 ...  1.7656519e-03
   2.6551452e-03  7.2244145e-03]
 [-3.1808448e-01 -6.1351520e-01 -2.0063773e-01 ...  4.4152541e-03
  -3.2643334e-03 -9.9789612e-03]] (23527, 4096)


In [5]:
np.save('C:/Users/Admin/Sushree_Codes/Sush/Results/train_gabor_feature_8_kernels_transformed_poly_4096.npy', train_gabor_feature_transformed)
